In [41]:
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from time import mktime
import base64
import requests
from pprint import pprint
import pandas as pd
import re
import json
import numpy as np

app_id = 'app id'
app_key = 'app key'
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' +                         'algorithm="hmac-sha1", ' +                         'headers="x-date", ' +                         'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }

a = Auth(app_id, app_key)

In [42]:
import logging
import datetime
import time
import os

In [43]:
#抓出現在日期
date_time=format(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S')

In [44]:
#file_path,log_path
file_path='C:/Users/lenovon/Desktop/bus/'
log_path='C:/Users/lenovon/Desktop/log_bus/'

In [45]:
#建立log，紀錄資訊
if os.path.isfile(log_path+date_time[:date_time.find(' ')]+'.txt'):
    logging.basicConfig(level=logging.WARNING,#控制檯列印的日誌級別
                        filename=log_path+date_time[:date_time.find(' ')]+'.txt',
                        filemode='a') #模式有w和a，w就是寫模式，每次都會重新寫日誌，覆蓋之前的日誌;a是append的意思
else:
    logging.basicConfig(level=logging.WARNING,#控制檯列印的日誌級別
                        filename=log_path+date_time[:date_time.find(' ')]+'.txt',
                        filemode='w') #模式有w和a，w就是寫模式，每次都會重新寫日誌，覆蓋之前的日誌;a是append的意思

In [46]:
#NewTaipei
try:
    response = requests.get('https://ptx.transportdata.tw/MOTC/v2/Bus/RealTimeByFrequency/City/NewTaipei?$top=10000&$format=JSON', 
                         headers=a.get_auth_header(),timeout=15)
except:
    logging.warning('Something happened:',exc_info=True)

In [47]:
temp_New=pd.DataFrame(response.json())

In [48]:
def RouteName_trans(x):
    return(x['Zh_tw'])
def SubRouteName_trans(x):
    return(x['Zh_tw'])
def BusPosition_trans(x):
    a=np.array(list(x.values()))
    a=np.round(a=a,decimals=4)
    return((a[0],a[1]))

In [49]:
temp_New['RouteName']=temp_New['RouteName'].map(RouteName_trans)
temp_New['SubRouteName']=temp_New['SubRouteName'].map(SubRouteName_trans)
temp_New['BusPosition']=temp_New['BusPosition'].map(BusPosition_trans)

In [50]:
#Taipei
try:
    response = requests.get('https://ptx.transportdata.tw/MOTC/v2/Bus/RealTimeByFrequency/City/Taipei?$top=10000&$format=JSON', 
                             headers=a.get_auth_header(),timeout=15)
except:
    logging.warning('Something happened:',exc_info=True)

In [51]:
temp_T=pd.DataFrame(response.json())

In [52]:
temp_T['RouteName']=temp_T['RouteName'].map(RouteName_trans)
temp_T['SubRouteName']=temp_T['SubRouteName'].map(SubRouteName_trans)
temp_T['BusPosition']=temp_T['BusPosition'].map(BusPosition_trans)

In [53]:
TEMP=pd.concat([temp_T,temp_New],ignore_index=True)

In [54]:
#修改時間格式
def revise_time(x):
    return datetime.datetime.strftime(datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S%z'),'%Y-%m-%d %H:%M:%S')

In [55]:
TEMP.columns

Index(['PlateNumb', 'OperatorID', 'RouteUID', 'RouteID', 'RouteName',
       'SubRouteUID', 'SubRouteID', 'SubRouteName', 'Direction', 'BusPosition',
       'Speed', 'Azimuth', 'DutyStatus', 'BusStatus', 'MessageType', 'GPSTime',
       'SrcUpdateTime', 'UpdateTime'],
      dtype='object')

In [56]:
TEMP['GPSTime']=TEMP['GPSTime'].map(revise_time)
TEMP['SrcUpdateTime']=TEMP['SrcUpdateTime'].map(revise_time)
TEMP['UpdateTime']=TEMP['UpdateTime'].map(revise_time)

In [57]:
#建立csv檔，以每天凌晨三點為界線
try:
    if (datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M:%S')-datetime.datetime.strptime(date_time[:date_time.find(' ')]+' 3:00:00','%Y-%m-%d %H:%M:%S')).total_seconds()>0: 
        if os.path.isfile(file_path+date_time[:date_time.find(' ')]+'.csv'):
            #有檔案
            TEMP.to_csv(file_path+date_time[:date_time.find(' ')]+'.csv',index=0,mode='a',header=0)
        else:
            #無檔案
            TEMP.to_csv(file_path+date_time[:date_time.find(' ')]+'.csv',index=0)
    else:
        date_time_r=datetime.datetime.strftime(datetime.datetime.strptime(date_time[:date_time.find(' ')],'%Y-%m-%d')-datetime.timedelta(days=1),'%Y-%m-%d')
        if os.path.isfile(file_path+date_time_r+'.csv'):
            #有檔案
            TEMP.to_csv(file_path+date_time_r+'.csv',index=0,mode='a',header=0)
        else:
            #無檔案
            TEMP.to_csv(file_path+date_time_r+'.csv',index=0)
except:
    logging.warning('File cannot establish well due to:',exc_info=True)

In [58]:
#建立csv檔，以每天凌晨三點為界線，寫進SQL資料庫
import pymysql
import MySQLdb
from sqlalchemy import create_engine
import os

In [59]:
TEMP['BusPosition']=TEMP['BusPosition'].astype('str')

In [60]:
conn = pymysql.connect(
    host='ip',
    port='port',
    user='username',
    password='password',
    db='dbname',
    charset='utf8mb4')
engine = create_engine('mysql+mysqldb://username:password@ip:port/dbname?charset=utf8mb4')
all_table=pd.read_sql_query('Show tables',conn)

In [61]:
all_table

,Tables_in_test_bus
0,T20200116


In [62]:
cursor=conn.cursor()

In [63]:
try:
    if (datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M:%S')-datetime.datetime.strptime(date_time[:date_time.find(' ')]+' 3:00:00','%Y-%m-%d %H:%M:%S')).total_seconds()>0: 
        date_time_r=format(datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M:%S'),'%Y%m%d')
        if ('T'+date_time_r) in all_table['Tables_in_test_bus'].values:
            #有檔案
            for i in range(0,TEMP.shape[0]):
                cursor.execute('insert into '+'T'+date_time_r+'(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s",%f,%f,"%s","%s","%s","%s","%s","%s")' % tuple(TEMP.columns.values.tolist()+TEMP.iloc[i,:].values.tolist()))
            conn.commit()
        else:
            date_time_r=format(datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M:%S'),'%Y%m%d')
            cursor.execute('create table '+'T'+date_time_r+' (aid int not NULL AUTO_INCREMENT,%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s float,%s float,%s varchar(255),%s varchar(255),%s varchar(255),%s DATETIME,%s DATETIME,%s DATETIME,primary key(aid))' % tuple(TEMP.columns.values.tolist()))
            conn.commit()
            #無檔案
            for i in range(0,TEMP.shape[0]):
                cursor.execute('insert into '+'T'+date_time_r+'(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s",%f,%f,"%s","%s","%s","%s","%s","%s")' % tuple(TEMP.columns.values.tolist()+TEMP.iloc[i,:].values.tolist()))
            conn.commit()
    else:
        date_time_r=datetime.datetime.strftime(datetime.datetime.strptime(date_time,'%Y-%m-%d %H:%M:%S')-datetime.timedelta(days=1),'%Y%m%d')
        if ('T'+date_time_r) in all_table['Tables_in_test_bus'].values:
            #有檔案
            for i in range(0,TEMP.shape[0]):
                cursor.execute('insert into '+'T'+date_time_r+'(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s",%f,%f,"%s","%s","%s","%s","%s","%s")' % tuple(TEMP.columns.values.tolist()+TEMP.iloc[i,:].values.tolist()))
            conn.commit()
        else:
            cursor.execute('create table '+'T'+date_time_r+' (aid int not NULL AUTO_INCREMENT,%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s varchar(255),%s float,%s float,%s varchar(255),%s varchar(255),%s varchar(255),%s DATETIME,%s DATETIME,%s DATETIME,primary key(aid))' % tuple(TEMP.columns.values.tolist()))
            conn.commit()
            #無檔案
            for i in range(0,TEMP.shape[0]):
                cursor.execute('insert into '+'T'+date_time_r+'(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) values ("%s","%s","%s","%s","%s","%s","%s","%s","%s","%s",%f,%f,"%s","%s","%s","%s","%s","%s")' % tuple(TEMP.columns.values.tolist()+TEMP.iloc[i,:].values.tolist()))
            conn.commit()
    conn.close()
except:
    logging.warning('File cannot establish well due to:',exc_info=True)
    conn.close()